<img src="img/mCIDaeNnb.png" alt="Logo CiDAEN" align="right">




<br><br><br>
<h2><font color="#00586D" size=4>Módulo 7</font></h2>



<h1><font color="#00586D" size=5>Usando Facebook Prophet en Python</font></h1>

<br><br><br>
<div style="text-align: right">
<font color="#00586D" size=3>Enrique González</font><br>
<font color="#00586D" size=3>Máster en Ciencia de Datos e Ingeniería de Datos en la Nube III </font><br>
<font color="#00586D" size=3>Universidad de Castilla-La Mancha</font>

</div>

---

<a id="indice"></a>
<h2><font color="#00586D" size=5>Índice</font></h2>


* [1. Introducción](#section1)
* [2. Modelo de pronóstico](#section2)
* [3. Evaluación de modelos](#section3)



In [1]:
import matplotlib.pyplot as plt

# Optimiza los gráficos para pantalla retina
%config InlineBackend.figure_format = 'retina'
# Por defecto usamos el backend inline
%matplotlib inline

# Oculta warnings
import warnings
warnings.simplefilter('ignore')

# La libreta ocupa así el 95% de la pantalla
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

---

<a id="section1"></a>
## <font color="#00586D"> 1. Introducción</font>
<br>

En esta práctica introducimos la librería Facebook Prophet para la creación de pronósticos. Esta librería tiene la ventaja de ser muy sencilla de usar y además incluir bastante funcionalidad útil para la visualización y evaluación de los resultados de pronóstico. Es ideal en series con grandes efectos estacionales, y cuando tenemos disponibles varios periodos estacionales. Asimismo, Prophet es robusto tanto a datos perdidos como a cambios en la tendencia. Todo esto, manteniendo una gran facilidad de uso lo que la convierte en una muy buena alternativa para obtener modelos de pronóstico en solo unas pocas líneas de código. 

Quizá lo más complicado de esta librería es instalarla. Para hacerlo os recomendamos seguir paso a paso las recomendaciones que dan desde el proyecto [para vuestro entorno específico](https://facebook.github.io/prophet/docs/installation.html#python), sobre todo en lo relativo a `pystan`.

Podéis conocer más sobre este modelo en la [página oficial del proyecto](https://facebook.github.io/prophet/). 

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Recordad__: Cread un `virtualenv` e instalad las librerias del `requirement.txt`para poder continuar con la práctica.  
</div>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

En esta práctica continuaremos con los conjuntos de datos de las prácticas anteriores. Para facilitar el acceso a la descripción de estos datos, os la incluyo a continuación. 

Son datos meteorológicos de dos estaciones meteorológicas españolas, la estación meteorológica del Retiro en Madrid y la estación meteorológica de La Coruña. Estos datos los podéis encontrar en `datos/meteo_retiro.csv`, con datos meteorológicos diarios desde el 1 de Enero de 1920 y `datos/meteo_coruña.csv`, con datos meteorológicos desde el 1 de Octubre de 1930.  

A continuación os indicamos la descripción de cada una de las columnas presentes en estas tablas:

- `FECHA`: Fecha en formato (aaaa-mm-dd) . Si lo abre con una hoja de cálculo, es posible que interprete el campo como fecha y lo vea en el formato habitual dd/mm/aaaa
- `INDICATIVO`: Identificador de Estación Meteorológica (Valor de 4 ó 5 caracteres)
- `NOMBRE`: Nombre de la Estación
- `PROVINCIA`: Provincia en la que se encuentra
- `ALTITUD`: Altitud (metros)
- `TMEDIA`: Temperatura media (ºC)
- `PRECIPITACION`: Precipitación (mm = l/m2)
- `TMIN`: Temperatura minima (ºC)
- `HORATMIN`: Hora de Temperatura mínima (hh:mm)
- `TMAX`: Temperatura Máxima (ºC)
- `HORATMAX`: Hora de Temperatura máxima (hh:mm)
- `DIR`: Dirección del viento (decenas de grado)
- `VELMEDIA`: Velocidad media (m/s)
- `RACHA`: Racha (m/s)
- `HORARACHA`: Hora Racha (hh:mm)
- `SOL`: Horas de Sol (horas)
- `PRESMAX`: Presión máxima (hPa)
- `HORAPRESMAX`: Hora de Presión Máxima (hh:mm)
- `PRESMIN`: Presión Mínima (hPa)
- `HORAPRESMIN`: Hora de Presión Mínima (hh:mm)


---

<a id="section2"></a>
## <font color="#00586D"> 2. Modelo de pronóstico</font>
<br>

Crear un modelo con Prophet es bastante sencillo. Solo hay que tener en cuenta algunas suposiciones que hace el modelo en cuanto a los datos de entrada. Para Prophet, el conjunto de datos de entrada tiene que tener dos coumnas:
- Una columna con nombre `ds`, que indica la marca de tiempo del dato. Los datos tendrán que tener una marca de tiempo siguiendo el formato `YYYY-MM-DD HH:MM:SS` y pueden contar con diferentes frecuencias: minutal, horaria, diaria, mensual...
- Una columna `y` con los valores de la serie temporal. 

Por lo tanto,el primer paso para poder utilizar Prophet es procesar la serie de datos. Vamos a comenzar cargando datos y dividiéndolos en entrenamiento y test como en prácticas anteriores. 

In [3]:
ts = pd.read_csv(
    './data/meteo_retiro.csv', 
    sep=';', 
    parse_dates=['FECHA'], 
    index_col='FECHA', 
    encoding='latin9'
).loc['2000':, 'TMEDIA']
n = len(ts)
split = 0.2 # % de datos para test
n_training = int((1-split)*n)
training = ts[:n_training]
test = ts[n_training:]
training.plot(figsize=(20,5), color='firebrick', ls='-');
test.plot(figsize=(20,5), color='silver', ls='-');
plt.legend(['Training', 'Test'])

FileNotFoundError: [Errno 2] No such file or directory: './data/meteo_retiro.csv'

Los datos tienen la siguiente forma. 

In [ ]:
training

Para transformarlos en el formato esperado por Prophet, tomaremos el índice temporal de los datos y crearemos una columna a partir de este. 

In [ ]:
training_prophet = training.reset_index().rename(columns={'FECHA': 'ds', 'TMEDIA': 'y'})
test_prophet = test.reset_index().rename(columns={'FECHA': 'ds', 'TMEDIA': 'y'})
training_prophet

Ya tenemos los datos preparados para ser utilizados por el modelo de Prophet. A continuación, entrenamos con los datos de _training_

In [ ]:
from prophet import Prophet
m = Prophet()
m.fit(training_prophet)

In [ ]:
test_prophet

Para probar el modelo podemos usar el método `predict`.

In [ ]:
forecast = m.predict(test_prophet)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

De esta forma, Prophet nos proporciona de forma muy sencilla tanto la estimación del valor de temperatura, como unos intervalos de confianza (por omisión, 80%) sobre esta predicción. Se puede modificar el tipo de intervalos de confianza generados en el momento de crear el modelo de prophet. 

In [ ]:
m = Prophet(interval_width=0.95)
m.fit(training_prophet)
forecast = m.predict(test_prophet)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

Podemos visualizar el resultado del pronóstico de la siguiente forma, reseteando el índice temporal. 

In [ ]:
training.plot(figsize=(20,5), color='firebrick', ls='-');
test.plot(figsize=(20,5), color='silver', ls='-');
pred = forecast.set_index('ds')['yhat']
pred.plot(figsize=(20,5), color='slateblue', ls='-');
plt.legend(['Training', 'Test', 'Forecast'])

O también podemos visualizar la predicción usando el método `plot` del modelo. 

In [ ]:
fig1 = m.plot(forecast)

Una vez entrenado nuestro modelo, normalmente nos interesará guardarlo. Para ello, la forma recomendada es usar las utilidades de serialización de Prophet y el paquete `json`. 

In [ ]:
import json
from prophet.serialize import model_to_json, model_from_json

with open('modelo_pronostico.json', 'w') as fout:
    json.dump(model_to_json(m), fout)  

Para cargarlo, podéis usar:

In [ ]:
with open('modelo_pronostico.json', 'r') as fin:
    m_saved = model_from_json(json.load(fin))  

In [ ]:
fig1 = m_saved.plot(forecast)

<div class="alert alert-block alert-warning">
    
<i class="fa fa-exclamation-circle" aria-hidden="true"></i>
__Importante__: Este modelo guardado de esta forma solo os servirá para hacer predicciones. No podréis utilizarlo (como veremos a continuación) con el método `cross_validation` ya que requeriría entrenar otra vez el modelo (y el objeto resultante de la serialización no lo permite)
</div>

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>

---

### <font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Ejercicio 1</font>

Crea un modelo de Prophet para el pronóstico de la temperatura máxima y mínima en Madrid usando los datos del retiro. Estas series están disponibles con los nombres TMAX y TMIN respectivamente. Múestra estos dos pronósticos en un gráfico junto con el pronóstico de la temperatura media creado anteriormente. 

In [ ]:
# SOLUCIÓN

In [ ]:
ts = pd.read_csv(
    './data/meteo_retiro.csv', 
    sep=';', 
    parse_dates=['FECHA'], 
    index_col='FECHA', 
    encoding='latin9'
).loc['2000':, 'TMAX']
n = len(ts)
split = 0.2 # % de datos para test
n_training = int((1-split)*n)
training_max_ts = ts[:n_training]
test_max_ts = ts[n_training:]
training_max = training_max_ts.reset_index().rename(columns={'FECHA': 'ds', 'TMAX': 'y'})
test_max = test_max_ts.reset_index().rename(columns={'FECHA': 'ds', 'TMAX': 'y'})

In [ ]:
ts = pd.read_csv(
    './data/meteo_retiro.csv', 
    sep=';', 
    parse_dates=['FECHA'], 
    index_col='FECHA', 
    encoding='latin9'
).loc['2000':, 'TMIN']
n = len(ts)
split = 0.2 # % de datos para test
n_training = int((1-split)*n)
training_min = ts[:n_training]
test_min = ts[n_training:]
training_min = training_min.reset_index().rename(columns={'FECHA': 'ds', 'TMIN': 'y'})
test_min = test_min.reset_index().rename(columns={'FECHA': 'ds', 'TMIN': 'y'})

In [ ]:
m_max = Prophet()
m_max.fit(training_max)
forecast = m_max.predict(test_max)
pred_max = forecast.set_index('ds')['yhat']

In [ ]:
m_min = Prophet()
m_min.fit(training_min)
forecast = m_min.predict(test_min)
pred_min = forecast.set_index('ds')['yhat']

In [ ]:
training.plot(figsize=(20,5), color='firebrick', ls='-');
test.plot(figsize=(20,5), color='silver', ls='-');
pred.plot(figsize=(20,5), color='slateblue', ls='-');
pred_max.plot(figsize=(20,5), color='darkorange', ls='-');
pred_min.plot(figsize=(20,5), color='darkgreen', ls='-');
plt.legend(['Training', 'Test', 'Forecast'])

In [ ]:
with open('modelo_max2.json', 'w') as fout:
    json.dump(model_to_json(m_max), fout)  

Resultado esperado:

![resultado](img/ej1.png)

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#00586D"></i></font></div>

--- 

---

<a id="section3"></a>
## <font color="#00586D"> 3. Evaluación de modelos </font>
<br>

Podemos evaluar los modelos de Prophet tal como veíamos en prácticas anteriores, usando `scikit-learn`. En este caso optamos por rellenar los NA para que no nos de problemas la función de evaluación, aunque también podríamos haberlos eliminado.   

In [ ]:
from sklearn.metrics import mean_absolute_error


mean_absolute_error(test.fillna(method='ffill'), pred.fillna(method='ffill'))

Como alternativa, Prophet nos proporciona un método para facilitarnos la evaluación por medio de validación cruzada temporal. Para usarla, deberemos definir 3 componentes. 
- `initial`: Número de periodos de datos inicial. A partir de este, se empezará la evaluación. Por ejemplo, si indicamos 2 años, se utilizarán esos dos años como conjunto de entrenamiento de partida que luego se irá aumentando a medida que se incrementa el punto de corte.  
- `horizon`: el horizonte que se pronosticará para realizar la evaluación. Por ejemplo, partiendo de los datos de entrenamiento de 2 años, se realiza un pronóstico de 1 año, sobre el que se aplicará la métrica de evaluación. 
- `period`: Marca cada cuantos periodos se realizará una evaluación. Esto es, si nuestro periodo inicial es de 2 años, nuestro horizonte es de 1 año y  nuestro periodo es de 6 meses, se evaluará, en el primer caso, un año desde los 2 años iniciales, en el segundo, un año entrenando con los 2 años iniciales más el periodo de 6 meses, y así se continuaría hasta acabar con los datos. 

Para utilizarlo, tendréis que usar el siguiente método. 

In [ ]:
from prophet.diagnostics import cross_validation

example_cv = cross_validation(m, initial='730 days', period='180 days', horizon = '365 days')
example_cv.head(100)

En este caso hemos usado, aproximadamente, dos años de periodo inicial, medio año de periodo y un horizonte de un año. Como resultado obtenemos la predicción para todos los horizontes, usando los datos previos a este para el entrenamiento. Como el periodo no coincide con el tamaño del horizonte, en muchos casos tendremos el doble de predicciones. Para visualizar los datos, vamos a agregar los datos repetidos por la media. 

In [ ]:
cv_pred = example_cv.set_index('ds').sort_index()['yhat']

In [ ]:
cv_pred['2004':].head()

In [ ]:
prediction = cv_pred.resample('D').mean()
prediction['2004':].head()

Vamos a visualizar el resultado. 

In [ ]:
training.plot(figsize=(20,5), color='silver', ls='-');
prediction.plot(figsize=(20,5), color='darkgreen', ls='-');
plt.legend(['Training', 'Forecast'])

Asimismo, Prophet nos permite obtener una serie de métricas de rendimiento teniendo en cuenta el número de días desde el último día de entrenamiento. Eso sí, para el cálculo de cada métrica utiliza una ventana deslizante sobre las predicciones, que tiene un tamaño del 10% de los datos. 


In [ ]:
from prophet.diagnostics import performance_metrics
df_p = performance_metrics(example_cv)
df_p.head()

Prophet también nos permite ver estos resultados en una gráfica. Por ejemplo, veamos el MAE. 

In [ ]:
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(example_cv, metric='mae')

De esta forma podemos ver si nuestro modelo pierde potencia a medida que avanzamos en el horizonte. En nuestro caso, sin embargo, parece que la calidad del pronóstico es más o menos constante.

<div style="text-align: right">
<a href="#indice"><font size=5><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#00586D"></i></font></a>
</div>

---

### <font color="#00586D"> <i class="fa fa-pencil-square-o" aria-hidden="true" style="color:#113D68"></i> Ejercicio 2</font>

Evalúa tu modelo de pronóstico para TMAX que creaste en el Ejercicio 1 mostrando como evoluciona el MAE a medida que se avanza en el horizonte. Obten también el MAE de tú pronóstico para todos los valores del horizonte, pero en este caso, no rellenes los valores NaN si no eliminalos de forma que puedas calcular el MAE solo con los valores adecuados. 

In [ ]:
#Solución

In [ ]:
from prophet.diagnostics import cross_validation

example_cv = cross_validation(m_max, initial='730 days', period='180 days', horizon = '365 days')
example_cv.head()

In [ ]:
training_max_ts.plot(figsize=(20,5), color='silver', ls='-');
example_cv.set_index('ds')['yhat'].sort_index().plot(figsize=(20,5), color='darkgreen', ls='-');
plt.legend(['Training', 'Forecast'])

In [ ]:
from prophet.plot import plot_cross_validation_metric
fig = plot_cross_validation_metric(example_cv, metric='mae')

In [ ]:
cv_pred = example_cv.set_index('ds').sort_index()['yhat']
prediction = cv_pred.resample('D').mean()

In [ ]:
first_date = prediction.index[0]
last_date = prediction.index[-1]
expected = training_max_ts[first_date:last_date]

In [ ]:
to_eval = pd.concat([expected, prediction], axis=1).dropna()

In [ ]:
mean_absolute_error(to_eval.iloc[:,0], to_eval.iloc[:,1])

Resultado esperado:
- MAE: 2.9758

![resultado](img/ej2.png)

<div style="text-align: right"><font size=4> <i class="fa fa-check-square-o" aria-hidden="true" style="color:#00586D"></i></font></div>

--- 



<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#00586D"></i> </font></div>